# Step 1: Imports and Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Adjust display settings to see all columns
pd.set_option('display.max_columns', None)

def load_data(file_path):
    """
    Loads data from csv or xlsx file types.
    """
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    elif file_path.endswith(('.xls', '.xlsx')):
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file type. Please provide a .csv or .xlsx file.")

# === USER INPUT ===
file_name = "your_dataset.csv"  # REPLACE with actual file name
# ==================

try:
    df = load_data(file_name)
    print(f"Data loaded successfully. Shape: {df.shape}")
except Exception as e:
    print(f"Error loading data: {e}")

# Step 2: Initial Exploration

In [ ]:
print("--- First 5 Rows ---")
display(df.head())

print("\n--- Data Info ---")
df.info()

print("\n--- Statistical Summary ---")
# 'include="all"' forces it to summarize strings/dates too, not just numbers
display(df.describe(include='all'))

# Step 3: Column Name Standardization
Inconsistent column names replacement:
- Strip whitespace (remove spaces before/after names).
- Convert to lower case.
- Replace spaces with underscores (snake_case).

In [ ]:
print("Original Columns:", df.columns.tolist())

# Clean columns: strip spaces, lowercase, replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

print("Cleaned Columns:", df.columns.tolist())

# Step 4: Handling Duplicates

In [ ]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows found: {duplicates}")

print(df.duplicated())

In [ ]:
if duplicates > 0:
    df = df.drop_duplicates()
    print("✅ Duplicates removed.")
else:
    print("No duplicates to remove.")

# Step 5: Missing Values

In [ ]:
# 1. Check for missing values
missing_percent = df.isnull().mean() * 100
print("--- Missing Value Percentage by Column ---")
print(missing_percent[missing_percent > 0])

# Visualizing missing data
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Data Heatmap (Yellow = Missing)")
plt.show()

# Warning: ACTION REQUIRED BELOW !!!!!

# Examples of how to fix specific columns (Uncomment and adapt as needed)

# Strategy A: Drop columns with too many missing values (e.g., > 50%)
# threshold = 50
# cols_to_drop = missing_percent[missing_percent > threshold].index
# df = df.drop(columns=cols_to_drop)

# Strategy B: Fill Numeric values with Median (safer than mean due to outliers)
# num_cols = df.select_dtypes(include=[np.number]).columns
# for col in num_cols:
#     df[col] = df[col].fillna(df[col].median())

# Strategy C: Fill Categorical values with 'Unknown'
# cat_cols = df.select_dtypes(include=['object']).columns
# for col in cat_cols:
#     df[col] = df[col].fillna('Unknown')

# Re-check
print("\nRemaining missing values:", df.isnull().sum().sum())

# Step 6: Data Type Conversion

In [ ]:
df.info()

In [ ]:
# Convert Date columns
# Identify columns that look like dates (e.g., 'date', 'time', 'created_at')
date_cols = ["aaa", "bbb"]

for col in date_cols:
    try:
        df[col] = pd.to_datetime(df[col])
        print(f"✅ Converted {col} to datetime.")
    except Exception as e:
        print(f"Could not convert {col}: {e}")

In [ ]:
# Clean Numeric Strings (e.g., "$1,200.50" -> 1200.50)
# This regex removes anything that isn't a digit or a decimal point
# df['price'] = df['price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
# df['price'] = pd.to_numeric(df['price'])

In [ ]:
# Check changes
df.info()

# Step 7: String Cleaning
- Strip whitespace.
- Standardize capitalization (Title Case or Lowercase)

In [ ]:
# Select all string (object) columns
str_cols = df.select_dtypes(include=['object']).columns


In [ ]:
# Exclude the 'city' column (and any others you want to protect)
# cols_to_exclude = ['city']
# str_cols = [col for col in str_cols if col not in cols_to_exclude]

In [ ]:
for col in str_cols:
    # Strip whitespace from start/end
    df[col] = df[col].str.strip()

    # Optional: Force logic (Uncomment if needed)
    # df[col] = df[col].str.lower()       # Make everything lowercase
    # df[col] = df[col].str.title()       # Make everything Title Case

print("Done")

# Step 8: Outlier Detection

In [ ]:
# Only check numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

    # Calculate bounds
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Column {col}: {len(outliers)} potential outliers detected.")

In [ ]:
# CONFIGURATION: Choose your method: 'cap', 'drop', or 'none'
outlier_method = 'cap'

for col in numeric_cols:
    # 1. Calculate Bounds
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 2. Count Outliers
    outlier_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    num_outliers = outlier_mask.sum()

    if num_outliers > 0:
        print(f"Column '{col}': Found {num_outliers} outliers.")

        # 3. Apply Chosen Method
        if outlier_method == 'drop':
            df = df[~outlier_mask]
            print(f"Action: Dropped {num_outliers} rows.")

        elif outlier_method == 'cap':
            # Replaces values outside bounds with the bounds themselves
            df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
            print(f"Action: Capped values at {lower_bound:.2f} and {upper_bound:.2f}")

        else:
            print("Action: None (View only)")
    else:
        print(f"Column '{col}': No outliers detected.")

print(f"\n Outlier processing complete. Final data shape: {df.shape}")

# Step 9: Logic & Sanity Checks

In [ ]:
# Example A: Check for negative values in columns that must be positive
cols_must_be_positive = [] # Add column names here, e.g. ['age', 'price']
for col in cols_must_be_positive:
    if col in df.columns:
        invalid_count = (df[col] < 0).sum()
        if invalid_count > 0:
            print(f"Found {invalid_count} negative values in {col}")
            # Fix: e.g., convert to absolute value
            # df[col] = df[col].abs()

In [ ]:
# Example B: Date Logic
# if 'start_date' in df.columns and 'end_date' in df.columns:
#     invalid_dates = df[df['start_date'] > df['end_date']]
#     print(f"Found {len(invalid_dates)} rows where Start Date > End Date")

# Step 10: Export File

In [ ]:
output_filename = "cleaned_data.csv"
df.to_csv(output_filename, index=False)

print(f"Data cleaning complete. Saved to {output_filename}")